# Clean Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

In [2]:
# Read the file into a DataFrame.
df = pd.read_csv("salary.csv")

df.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Handle Missing Data

In [3]:
# Print the number of NaN values per column.
print("NaN values per column before data cleaning: ")
print(df.isnull().sum())

# Print the number of ' ?' values per column since missing values are ' ?' in this dataset.
print("' ?' values per column before data cleaning: ")
print((df == ' ?').sum())

NaN values per column before data cleaning: 
age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
salary            0
dtype: int64
' ?' values per column before data cleaning: 
age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
salary               0
dtype: int64


In [4]:
# Replace '?' with NaN for subsequent processing.
df.replace(' ?', np.nan, inplace=True)

# Fill numeric columns with their mean.
numeric_columns = df.select_dtypes(include=[np.number]).columns
df[numeric_columns] = df[numeric_columns].apply(lambda col: col.fillna(col.mean()))

# Retrieve categorical columns.
categorical_cols = df.select_dtypes(include=['object']).columns

# Convert categorical columns to numerical encoding.
df_categorical = df[categorical_cols].apply(lambda col: col.astype('category').cat.codes)

# Apply KNN imputation to categorical data.
knn_imputer = KNNImputer(n_neighbors=5)
df_categorical_imputed = pd.DataFrame(knn_imputer.fit_transform(df_categorical), columns=categorical_cols)

# Convert back the imputed categorical columns to original categories.
for col in categorical_cols:
    df_categorical_imputed[col] = df_categorical_imputed[col].round().astype(int)  # Round to nearest integer
    df_categorical_imputed[col] = pd.Categorical.from_codes(
        df_categorical_imputed[col], df[col].astype('category').cat.categories
    )

# Assign imputed categorical values back to original dataframe.
df[categorical_cols] = df_categorical_imputed

In [5]:
# Print the number of missing values per column after data cleaning.
print("Missing values per column after data cleaning: ")
print(df.isnull().sum())

# Print the number of ' ?' values per column since missing values are ' ?' in this dataset.
print("Missing values per column after data cleaning: ")
print((df == ' ?').sum())

Missing values per column after data cleaning: 
age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
salary               0
dtype: int64
Missing values per column after data cleaning: 
age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
salary            0
dtype: int64


## Handle Duplicated Data

In [6]:
# Print the number of duplicated rows.
print(f"Number of duplicated rows before removal: {df.duplicated().sum()}")

Number of duplicated rows before removal: 24


In [7]:
# Remove the duplicated rows.
df.duplicated() 
df = df.drop_duplicates()

In [8]:
# Confirm that the duplicated rows have been removed.
print(f"Number of duplicated rows after removal: {df.duplicated().sum()}")

Number of duplicated rows after removal: 0


## Convert Data Types

In [9]:
# Convert the data to correct types.
df['age'] = df['age'].astype('int')
df['workclass'] = df['workclass'].astype('category')
df['fnlwgt'] = df['fnlwgt'].astype('int')
df['education'] = df['education'].astype('category')
df['education-num'] = df['education-num'].astype('int')
df['marital-status'] = df['marital-status'].astype('category')
df['occupation'] = df['occupation'].astype('category')
df['relationship'] = df['relationship'].astype('category')
df['race'] = df['race'].astype('category')
df['sex'] = df['sex'].astype('category')
df['capital-gain'] = df['capital-gain'].astype('int')
df['capital-loss'] = df['capital-loss'].astype('int')
df['hours-per-week'] = df['hours-per-week'].astype('int')
df['native-country'] = df['native-country'].astype('category')
df['salary'] = df['salary'].astype('category')

# Convert the salary column to Boolean values.
df.rename(columns={'salary': 'salary-greater-50k'}, inplace=True)
df['salary-greater-50k'] = df['salary-greater-50k'].str.strip().apply(lambda x: True if x == '>50K' else False)

## Handle Outliers

In [10]:
# Identify the numerical columns, excluding the 'salary' column.
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
if 'salary' in numerical_cols:
    numerical_cols.remove('salary')

# Process each numerical column.
for col in numerical_cols:
    # Skip 'capital-gain' and 'capital-loss' to handle them separately.
    if col in ['capital-gain', 'capital-loss', 'hours-per-week']:
        continue
    
    # Calculate Q1, Q3, and IQR.
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    # Define lower and upper bounds for outliers using IQR.
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Set custom bounds for 'age' column.
    if col == 'age':
        lower_bound = 17

        # Log information about the calculated bounds and potential outliers.
    print(f"Column: {col}")
    print(f"  Lower bound: {lower_bound}")
    print(f"  Upper bound: {upper_bound}")
    print(f"  Outliers found: {((df[col] < lower_bound) | (df[col] > upper_bound)).sum()}")
    
    # Cap outlier values at the calculated bounds.
    df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)

# Process 'capital-gain' and 'capital-loss' columns separately.
for col in ['capital-gain', 'capital-loss']:
    # Filter out zero values for IQR calculation to avoid the zero values dominating.
    non_zero_col = df[df[col] > 0][col]

    # Calculate Q1, Q3, and IQR for non-zero values only.
    Q1 = non_zero_col.quantile(0.25)
    Q3 = non_zero_col.quantile(0.75)
    IQR = Q3 - Q1
    
    # Define lower and upper bounds for outliers using IQR.
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Log information about the bounds and potential outliers.
    print(f"Column: {col}")
    print(f"  Lower bound: {lower_bound}")
    print(f"  Upper bound: {upper_bound}")
    print(f"  Outliers found: {((df[col] < lower_bound) | (df[col] > upper_bound)).sum()}")

    # Apply capping only to non-zero values.
    df.loc[df[col] > 0, col] = df.loc[df[col] > 0, col].clip(lower=lower_bound, upper=upper_bound)

Column: age
  Lower bound: 17
  Upper bound: 78.0
  Outliers found: 142
Column: fnlwgt
  Lower bound: -60922.0
  Upper bound: 415742.0
  Outliers found: 993
Column: education-num
  Lower bound: 4.5
  Upper bound: 16.5
  Outliers found: 1193
Column: capital-gain
  Lower bound: -12598.5
  Upper bound: 30093.5
  Outliers found: 166
Column: capital-loss
  Lower bound: 1214.5
  Upper bound: 2434.5
  Outliers found: 31125


/var/folders/_4/jt_csdv54z50v8hfjy064zb00000gn/T/ipykernel_33460/481881657.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 2174. 14084.  5178. ... 15020.  1086. 15024.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[col] > 0, col] = df.loc[df[col] > 0, col].clip(lower=lower_bound, upper=upper_bound)
/var/folders/_4/jt_csdv54z50v8hfjy064zb00000gn/T/ipykernel_33460/481881657.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2042.  1408.  1902.  ... 1669.  1977.  1214.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[col] > 0, col] = df.loc[df[col] > 0, col].clip(lower=lower_bound, upper=upper_bound)


### Save Cleaned Data

In [11]:
# Save cleaned data to salary_cleaned.csv.
df.to_csv("salary_cleaned.csv", index=False)